<pre>
changelog of real files:
    * modified nodule_parser.py 'file_to_text' function to force utf-8 and ignore non-utf8 characters
    * passed 'label' column out of 'learner' as truth_marking (if vetting == True)
    
* converted nodule_learner.py to allow for multithreading/multiprocessing
   * re-factored predict to NOT be memory limited (chunks data through and joins it up as it streams out)
* added HCA/OBX file readers (aka extractor) -nodule_parser.py
</pre>


# setup everything

In [18]:
import nodule_parser
import nodule_learner
import nodule_extractor
import imp
imp.reload(nodule_parser)
imp.reload(nodule_learner)
imp.reload(nodule_extractor)
import pandas as pd
import json

import re
import numpy as np
import traceback
import warnings
import sys
from datetime import datetime
import os
from math import floor

from queue import Queue
from threading import Thread, Lock
import sqlite3

import boto3


def warn_with_traceback(message, category, filename, lineno, file=None, line=None):
    log = file if hasattr(file,'write') else sys.stderr
    traceback.print_stack(file=log)
    log.write(warnings.formatwarning(message, category, filename, lineno, line))

warnings.showwarning = warn_with_traceback

nl = nodule_learner.NoduleLearner(cache_size=1000)

# download from bucket #

// to output how many lines are in each file accross a directory
<pre>
find . -name '*.txt' | xargs wc -l
</pre>

In [ ]:
# GB = 1024 ** 3
# config_s3 = boto3.s3.transfer.TransferConfig(
#     multipart_threshold=1 * GB,
#     max_concurrency=10,
#     num_download_attempts=10,
#     multipart_chunksize=1 * GB,
#     max_io_queue=10000
# )

def download_dir(resource, file_prefix, local='/tmp', bucket='your_bucket'):
    '''
    file_prefix - str : what the 'items' in the bucket must start with ( filter )
    local - str : local directory (path from here) where to download data to
    bucket - str : where to pull data from
    '''
    files_out = []
    client = resource.meta.client
    paginator = client.get_paginator('list_objects')
    for result in paginator.paginate(Bucket=bucket, Delimiter='/', Prefix=file_prefix, ):
        if result.get('CommonPrefixes') is not None:
            for subdir in result.get('CommonPrefixes'):
                download_dir(client, resource, subdir.get('Prefix'), local, bucket)
        if result.get('Contents') is not None:
            for file in result.get('Contents'):
                if file.get('Key')[-1] != "/":
                    print("saved file: %s" % file.get('Key'))
                    files_out.append( file.get('Key') )
                    if not os.path.exists(os.path.dirname(local + os.sep + file.get('Key'))):
                         os.makedirs(os.path.dirname(local + os.sep + file.get('Key')))
                    resource.meta.client.download_file( bucket, file.get('Key'),
                                                       local + os.sep + file.get('Key'),
#                                                       Config=config_s3
                                                      )
                    #
# ORU-OBX_All_Campuses/HCA/ParsedHourlyFiles/
# ORU-OBX_All_Campuses/HCA/ParsedDailyFiles/
# ORU-OBX_All_Campuses/HCA/ParsedOBXFiles/DailyOBX
# ORU-OBX_All_Campuses/HCA/ParsedOBXFiles/HourlyOBX
resource = boto3.resource('s3')
download_dir(resource, 'ORU-OBX_All_Campuses/HCA/ParsedOBXFiles/HourlyOBX/', 
             local='../OBX_8_17', bucket='dicom-images-virtualdirect')

# Load data
Caveats:<br>
<ul>
    <li>only reads 1 level deep - make sure folder/file structure works</li>
    <li>only parses .txt files </li>
    <li>commented out code will limit parser to read the firs 20k files per dir</li>
</ul>

In [ ]:
%prun nl.transform_predict(file=fileName, probability=False, vetting=True, hca=True)

In [30]:
# conn = sqlite3.Connection("predictions_20180725.sqlite")
# pd.read_sql("SELECT * FROM prediction", conn)
# patient_data = pd.read_sql("SELECT * FROM patient_data", nl.conn)
# patient_data

imp.reload(nodule_parser)
imp.reload(nodule_learner)
imp.reload(nodule_extractor)
nl = nodule_learner.NoduleLearner(cache_size=0)
# fileName = "../HCA_ParsedHourlyFiles/ORU-OBX_All_Campuses/HCA//ParsedHourlyFiles/edhlungdirectdata_hourly_20180721233125.txt"
# smaller  121929
# fileName = "../HCA_ParsedHourlyFiles/ORU-OBX_All_Campuses/HCA//ParsedHourlyFiles/EDHLungDirectData_Hourly_20180314033110.txt"
# medium 246,401 lines
# fileName = "../HCA_ParsedHourlyFiles/ORU-OBX_All_Campuses/HCA//ParsedHourlyFiles/EDHLungDirectData_Hourly_20180319023140.txt"
# # larger ~ 740,000 lines
# fileName = "../HCA_ParsedHourlyFiles/ORU-OBX_All_Campuses/HCA//ParsedHourlyFiles/EDHLungDirectData_Hourly_20180317083059.txt"


# fileName ="../OBX_8_17/ORU-OBX_All_Campuses/HCA/ParsedOBXFiles/HourlyOBX/EDHLungDirectData_Hourly_obx_201802211330.txt"
## had error 
# fileName = "../HCA_ParsedHourlyFiles/ORU-OBX_All_Campuses/HCA//ParsedHourlyFiles/edhlungdirectdata_hourly_20180721233125.txt"
fileName = "../OBX_8_17/ORU-OBX_All_Campuses/HCA/ParsedOBXFiles/HourlyOBX/edhlungdirectdata_hourly_obx_201808141431.txt"
### failed - prediction == 0 but it is == 1
# fileName ="../OBX_8_17/ORU-OBX_All_Campuses/HCA/ParsedOBXFiles/HourlyOBX/edhlungdirectdata_hourly_obx_201808160831.txt"


fileName ="../OBX_8_17/ORU-OBX_All_Campuses/HCA/ParsedOBXFiles/HourlyOBX/edhlungdirectdata_hourly_obx_201808080931.txt"

# fileName ="../data-20180629/CCF/16655445_20170805161155633.txt"
# df = 
df =pd.DataFrame( json.loads(
                    nl.transform_predict(
                        file=fileName,
                        probability=True,
                        vetting=True, 
                        hca=True
                    )) )
df
# nl.features_df
# from nodule_parser import normalize,no_nodule_checker,is_vetted_df
# file =fileName
# file_df = pd.read_csv(file, sep="|", header=None, dtype=np.object,
#                       usecols=[0,4,8,9,20,21,34])


,directory,evidence,filename,is_nodule,max_nodule_change,max_nodule_location,max_nodule_lung,max_nodule_size,nodule_location,nodule_lung,...,nodule_size,nodule_size_numeric,phrase_counter,pid,prediction,prev_max_date,probability,report_date,text,truth_marking
0,NaN,specifically the current exam the right hilar ...,../OBX_8_17/ORU-OBX_All_Campuses/HCA/ParsedOBX...,NaN,0.0,,,15.0,NaN,NaN,...,NaN,NaN,NaN,E00009915844,1,07/08/2018,0.820,07/08/2018,patient name: mitchaelmarilyn jane unit no: e ...,-1
1,NaN,lung windows demonstrate numerous noncalcific ...,../OBX_8_17/ORU-OBX_All_Campuses/HCA/ParsedOBX...,NaN,0.0,,,0.0,NaN,NaN,...,NaN,NaN,NaN,E00009948728,0,,0.670,08/06/2018,patient name: huntnorma jean unit no: e exams:...,-1
2,NaN,findings: ct abdomen: the included lung bases ...,../OBX_8_17/ORU-OBX_All_Campuses/HCA/ParsedOBX...,NaN,0.0,,,0.0,NaN,NaN,...,NaN,NaN,NaN,W00051018745,0,,0.633,08/05/2018,patient name: fisherteresa questa unit no: w e...,-1
3,NaN,patient name: hollandjohn paul unit no: w exam...,../OBX_8_17/ORU-OBX_All_Campuses/HCA/ParsedOBX...,NaN,0.0,right upper lobe,right,6.0,NaN,NaN,...,NaN,NaN,NaN,W00051000698,1,08/02/2018,0.718,08/02/2018,patient name: hollandjohn paul unit no: w exam...,-1
4,NaN,findings: ct abdomen: the included lung bases ...,../OBX_8_17/ORU-OBX_All_Campuses/HCA/ParsedOBX...,NaN,0.0,,,0.0,NaN,NaN,...,NaN,NaN,NaN,W00051011908,0,,0.818,08/03/2018,patient name: hernandezbrittany leigh unit no:...,-1
5,NaN,numerous bilateral pulmonary nodules are noted...,../OBX_8_17/ORU-OBX_All_Campuses/HCA/ParsedOBX...,NaN,0.0,,,0.0,NaN,NaN,...,NaN,NaN,NaN,W00051014503,0,,0.649,08/04/2018,patient name: shawbrian thomas unit no: w exam...,-1
6,NaN,findings: pulmonary arteries: optimal pulmonar...,../OBX_8_17/ORU-OBX_All_Campuses/HCA/ParsedOBX...,NaN,0.0,,,0.0,NaN,NaN,...,NaN,NaN,NaN,W00051015811,0,,0.572,08/04/2018,patient name: hunterholly anne unit no: w exam...,-1
7,NaN,contrast enhanced thin section helical ct imag...,../OBX_8_17/ORU-OBX_All_Campuses/HCA/ParsedOBX...,NaN,0.0,,,0.0,NaN,NaN,...,NaN,NaN,NaN,W00051010576,0,,0.623,08/03/2018,patient name: dinhsuu van unit no: w exams: cp...,-1
8,NaN,no large prevertebral or paraspinal soft tissu...,../OBX_8_17/ORU-OBX_All_Campuses/HCA/ParsedOBX...,NaN,0.0,,,0.0,NaN,NaN,...,NaN,NaN,NaN,W00051015297,0,,0.664,08/04/2018,patient name: dickbrendan m unit no: w exams: ...,-1
9,NaN,no large prevertebral or paraspinal soft tissu...,../OBX_8_17/ORU-OBX_All_Campuses/HCA/ParsedOBX...,NaN,0.0,left lower lobe,left,12.0,NaN,NaN,...,NaN,NaN,NaN,W00051025540,1,08/06/2018,0.606,08/06/2018,patient name: whiteandrew m unit no: w exams: ...,-1


In [31]:
df.max_nodule_lung.value_counts()

left     7
right    4
         2
Name: max_nodule_lung, dtype: int64

In [64]:
# E00009915844=
# extracted=  [{"nodule_phrase": "findings: pulmonary arteries: examination is diagnostic, with pulmonary arterial opacification to the segmental level", "nodule_size": "", "nodule_location": "", "nodule_size_numeric": 0, "nodule_lung": "", "is_nodule": 1, "max_nodule_size": "1.5 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 15.0}, {"nodule_phrase": "there is extensive airspace opacification atelectasis involving the lateral middle lobe, soft tissue density throughout the right middle lobe airway", "nodule_size": "", "nodule_location": "right middle lobe", "nodule_size_numeric": 0, "nodule_lung": "right", "is_nodule": 1, "max_nodule_size": "1.5 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 15.0}, {"nodule_phrase": "though not well circumscribed, this mass approximately measures 3.1 x 2.3 x 1.5 cm image 76 series 4, image 162 series 5", "nodule_size": "1.5 cm", "nodule_location": "", "nodule_size_numeric": 15.0, "nodule_lung": "", "is_nodule": 1, "max_nodule_size": "1.5 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 15.0}, {"nodule_phrase": "multiple perivascular nodular parenchymal densities adjacent to the main consolidative region raise concern for endobronchial tumor spread image 72 series 4", "nodule_size": "", "nodule_location": "", "nodule_size_numeric": 0, "nodule_lung": "", "is_nodule": 1, "max_nodule_size": "1.5 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 15.0}, {"nodule_phrase": "no pleural effusion, nodularity, or pneumothorax is seen", "nodule_size": "", "nodule_location": "", "nodule_size_numeric": 0, "nodule_lung": "", "is_nodule": 1, "max_nodule_size": "1.5 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 15.0}, {"nodule_phrase": "pulmonary findings concerning for central endobronchial mass within the right middle lobe airway, with postobstructive consolidation", "nodule_size": "", "nodule_location": "right middle lobe", "nodule_size_numeric": 0, "nodule_lung": "right", "is_nodule": 1, "max_nodule_size": "1.5 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 15.0}, {"nodule_phrase": "perivascular nodular densities in this region raise concern for endobronchial tumor spread", "nodule_size": "", "nodule_location": "", "nodule_size_numeric": 0, "nodule_lung": "", "is_nodule": 1, "max_nodule_size": "1.5 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 15.0}, {"nodule_phrase": "noncalcified nodule in the left upper lobe", "nodule_size": "", "nodule_location": "left upper lobe", "nodule_size_numeric": 0, "nodule_lung": "left", "is_nodule": 1, "max_nodule_size": "1.5 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 15.0}, {"nodule_phrase": "nonspiculated pulmonary nodule is noted within the left upper lobe", "nodule_size": "", "nodule_location": "left upper lobe", "nodule_size_numeric": 0, "nodule_lung": "left", "is_nodule": 1, "max_nodule_size": "1.5 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 15.0}]
extracted=  [{"nodule_phrase": "findings: pulmonary arteries: examination is diagnostic with pulmonary arterial opacification to the segmental level", "nodule_size": "", "nodule_location": "", "nodule_size_numeric": 0, "nodule_lung": "", "is_nodule": 1, "max_nodule_size": "1.5 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 15.0}, {"nodule_phrase": "there is extensive airspace opacification atelectasis involving the lateral middle lobe soft tissue density throughout the right middle lobe airway", "nodule_size": "", "nodule_location": "right middle lobe", "nodule_size_numeric": 0, "nodule_lung": "right", "is_nodule": 1, "max_nodule_size": "1.5 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 15.0}, {"nodule_phrase": "though not well circumscribed this mass approximately measures 3.1 x 2.3 x 1.5 cm image 76 series 4 image 162 series 5", "nodule_size": "1.5 cm", "nodule_location": "", "nodule_size_numeric": 15.0, "nodule_lung": "", "is_nodule": 1, "max_nodule_size": "1.5 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 15.0}, {"nodule_phrase": "multiple perivascular nodular parenchymal densities adjacent to the main consolidative region raise concern for endobronchial tumor spread image 72 series 4", "nodule_size": "", "nodule_location": "", "nodule_size_numeric": 0, "nodule_lung": "", "is_nodule": 1, "max_nodule_size": "1.5 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 15.0}, {"nodule_phrase": "a noncalcified nonspiculated pulmonary nodule is noted within the left upper lobe measuring 6 mm image 58 series 4", "nodule_size": "6 mm", "nodule_location": "left upper lobe", "nodule_size_numeric": 6.0, "nodule_lung": "left", "is_nodule": 1, "max_nodule_size": "1.5 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 15.0}, {"nodule_phrase": "no pleural effusion nodularity or pneumothorax is seen", "nodule_size": "", "nodule_location": "", "nodule_size_numeric": 0, "nodule_lung": "", "is_nodule": 1, "max_nodule_size": "1.5 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 15.0}, {"nodule_phrase": "pulmonary findings concerning for central endobronchial mass within the right middle lobe airway with postobstructive consolidation", "nodule_size": "", "nodule_location": "right middle lobe", "nodule_size_numeric": 0, "nodule_lung": "right", "is_nodule": 1, "max_nodule_size": "1.5 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 15.0}, {"nodule_phrase": "perivascular nodular densities in this region raise concern for endobronchial tumor spread", "nodule_size": "", "nodule_location": "", "nodule_size_numeric": 0, "nodule_lung": "", "is_nodule": 1, "max_nodule_size": "1.5 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 15.0}, {"nodule_phrase": "noncalcified nodule in the left upper lobe", "nodule_size": "", "nodule_location": "left upper lobe", "nodule_size_numeric": 0, "nodule_lung": "left", "is_nodule": 1, "max_nodule_size": "1.5 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 15.0}]
extracted=  [{"nodule_phrase": "the left thyroid gland contains a heterogeneous nodule inferiorly which measures up to 2.7 cm series 2 image 8", "nodule_size": "2.7 cm", "nodule_location": "", "nodule_size_numeric": 27.0, "nodule_lung": "", "is_nodule": 1, "max_nodule_size": "2.7 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 27.0}, {"nodule_phrase": "there are multiple pulmonary nodules scattered throughout both lungs", "nodule_size": "", "nodule_location": "", "nodule_size_numeric": 0, "nodule_lung": "", "is_nodule": 1, "max_nodule_size": "2.7 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 27.0}, {"nodule_phrase": "for reference a pulmonary nodule in the right upper lobe measures 1.7 x 1.4 cm series 6 image 20 and a nodule in the left lower lobe measures 1.3 x 1.1 cm series 6 image 45", "nodule_size": "1.7 cm", "nodule_location": "left lower lobe", "nodule_size_numeric": 17.0, "nodule_lung": "left", "is_nodule": 1, "max_nodule_size": "2.7 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 27.0}, {"nodule_phrase": "abdomen and pelvis: there is a large heterogeneously enhancing mass replacing the right kidney", "nodule_size": "", "nodule_location": "", "nodule_size_numeric": 0, "nodule_lung": "", "is_nodule": 1, "max_nodule_size": "2.7 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 27.0}, {"nodule_phrase": "the mass is indistinguishable from the right adrenal gland and there is soft tissue stranding about the mass which displaces the ascending colon and small bowel loops to the left", "nodule_size": "", "nodule_location": "", "nodule_size_numeric": 0, "nodule_lung": "", "is_nodule": 1, "max_nodule_size": "2.7 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 27.0}, {"nodule_phrase": "heterogeneously enhancing right renal mass with tumor thrombus extending into the right renal veins inferior vena cava and right atrium", "nodule_size": "", "nodule_location": "", "nodule_size_numeric": 0, "nodule_lung": "", "is_nodule": 1, "max_nodule_size": "2.7 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 27.0}, {"nodule_phrase": "innumerable bilateral noncalcified pulmonary nodules and hyperenhancing liver lesions are compatible with pulmonary and liver metastasis", "nodule_size": "", "nodule_location": "", "nodule_size_numeric": 0, "nodule_lung": "", "is_nodule": 1, "max_nodule_size": "2.7 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 27.0}, {"nodule_phrase": "heterogeneous left inferior thyroid lobe nodule measuring 2.7 cm 6", "nodule_size": "2.7 cm", "nodule_location": "", "nodule_size_numeric": 27.0, "nodule_lung": "", "is_nodule": 1, "max_nodule_size": "2.7 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 27.0}, {"nodule_phrase": "the left thyroid gland contains a heterogeneous nodule inferiorly which measures up to 2.7 cm series 2 image 8", "nodule_size": "2.7 cm", "nodule_location": "", "nodule_size_numeric": 27.0, "nodule_lung": "", "is_nodule": 1, "max_nodule_size": "2.7 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 27.0}, {"nodule_phrase": "there are multiple pulmonary nodules scattered throughout both lungs", "nodule_size": "", "nodule_location": "", "nodule_size_numeric": 0, "nodule_lung": "", "is_nodule": 1, "max_nodule_size": "2.7 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 27.0}, {"nodule_phrase": "for reference a pulmonary nodule in the right upper lobe measures 1.7 x 1.4 cm series 6 image 20 and a nodule in the left lower lobe measures 1.3 x 1.1 cm series 6 image 45", "nodule_size": "1.7 cm", "nodule_location": "left lower lobe", "nodule_size_numeric": 17.0, "nodule_lung": "left", "is_nodule": 1, "max_nodule_size": "2.7 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 27.0}, {"nodule_phrase": "abdomen and pelvis: there is a large heterogeneously enhancing mass replacing the right kidney", "nodule_size": "", "nodule_location": "", "nodule_size_numeric": 0, "nodule_lung": "", "is_nodule": 1, "max_nodule_size": "2.7 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 27.0}, {"nodule_phrase": "the mass is indistinguishable from the right adrenal gland and there is soft tissue stranding about the mass which ed derby name: vannesseuvanda annette n rock road hp: 316-250- age: 58 s:f derby kansas dob: 07 29 loc: w", "nodule_size": "", "nodule_location": "", "nodule_size_numeric": 0, "nodule_lung": "", "is_nodule": 1, "max_nodule_size": "2.7 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 27.0}, {"nodule_phrase": "heterogeneously enhancing right renal mass with tumor thrombus extending into the right renal veins inferior vena cava and right atrium", "nodule_size": "", "nodule_location": "", "nodule_size_numeric": 0, "nodule_lung": "", "is_nodule": 1, "max_nodule_size": "2.7 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 27.0}, {"nodule_phrase": "innumerable bilateral noncalcified pulmonary nodules and hyperenhancing liver lesions are compatible with pulmonary and liver metastasis", "nodule_size": "", "nodule_location": "", "nodule_size_numeric": 0, "nodule_lung": "", "is_nodule": 1, "max_nodule_size": "2.7 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 27.0}, {"nodule_phrase": "heterogeneous left inferior thyroid lobe nodule measuring 2.7 cm 6", "nodule_size": "2.7 cm", "nodule_location": "", "nodule_size_numeric": 27.0, "nodule_lung": "", "is_nodule": 1, "max_nodule_size": "2.7 cm", "max_nodule_location": "", "max_nodule_lung": "", "max_nodule_size_numeric": 27.0}]

from pprint import pprint
len(extracted)
pprint( extracted )
# A noncalcified, nonspiculated pulmonary nodule is noted within the left upper lobe, measuring 6 mm

[{'is_nodule': 1,
  'max_nodule_location': '',
  'max_nodule_lung': '',
  'max_nodule_size': '2.7 cm',
  'max_nodule_size_numeric': 27.0,
  'nodule_location': '',
  'nodule_lung': '',
  'nodule_phrase': 'the left thyroid gland contains a heterogeneous nodule '
                   'inferiorly which measures up to 2.7 cm series 2 image 8',
  'nodule_size': '2.7 cm',
  'nodule_size_numeric': 27.0},
 {'is_nodule': 1,
  'max_nodule_location': '',
  'max_nodule_lung': '',
  'max_nodule_size': '2.7 cm',
  'max_nodule_size_numeric': 27.0,
  'nodule_location': '',
  'nodule_lung': '',
  'nodule_phrase': 'there are multiple pulmonary nodules scattered throughout '
                   'both lungs',
  'nodule_size': '',
  'nodule_size_numeric': 0},
 {'is_nodule': 1,
  'max_nodule_location': '',
  'max_nodule_lung': '',
  'max_nodule_size': '2.7 cm',
  'max_nodule_size_numeric': 27.0,
  'nodule_location': 'left lower lobe',
  'nodule_lung': 'left',
  'nodule_phrase': 'for reference a pulmonary nodule

In [46]:
re.sub(r'\s+', " ", """asdasdd ^      
sdfsdf dsf  sdf""".replace("^","").replace("\n"," "))
# .replace("^","").replace("\n"," ").replace(r"\s+","\s")
# re.sub?

'asdasdd sdfsdf dsf sdf'

# RUN THIS

In [2]:
## threaded version
# top = "../data-20180319/"

top = "../HCA_ParsedHourlyFiles/ORU-OBX_All_Campuses/HCA/"
top ="../OBX_8_17/ORU-OBX_All_Campuses/HCA/ParsedOBXFiles/"

max_files_per_dir = 8  #read first 20k files per folder --- uncomend usage below if you need

## if Vetting => save to a sql db
vetting = True
hca = True

if vetting: #stupid but wtvr
    prediction_sql = "predictions_20180822.sqlite"
    conn = sqlite3.Connection(prediction_sql)
    b = pd.DataFrame([{'directory': '', 'filename': '', 'max_nodule_change': '', 'text':'',
     'max_nodule_location': '','max_nodule_lung': '', 'max_nodule_size': '', 'pid': '','is_nodule':'',
     'prediction': '','prev_max_date': '', 'probability': '', 'report_date': '','truth_marking': '', 'phrase_counter':'',
      'nodule_location':'', 'nodule_lung':'', 'nodule_phrase':'','nodule_size_numeric':'','evidence': '', 'nodule_size':''}])
    b.to_sql("prediction", conn, if_exists='replace', chunksize=1000)

predictions = []
start_time = datetime.now()

from multiprocessing import Process
from multiprocessing import JoinableQueue as Queue

fileNames_q = Queue()
results = Queue()
sql_lock = Lock()

print("creating list of files to run through parrell parsing ")
for folder in os.listdir(top):
    counter = 0
    for file in sorted(os.listdir(top + folder)):
        if file.endswith("txt") and "EDHLungDirectData_Daily" not in file and "DailyOBX" not in folder:
            fileNames_q.put( top + folder + "/" + file )
            counter+=1
#             if counter>5 and counter<20:
#                 print(file)
#             if counter > max_files_per_dir:
#                 break
                
print("%d files to parse " % fileNames_q.qsize() )
def parse_files( fileNames_q, results, sql_lock ):
    counter = 0
    nl = nodule_learner.NoduleLearner(cache_size=1, thread_lock=sql_lock)
    while True:
        counter+=1
        fileName = fileNames_q.get()
#         try:
        if vetting:
            prediction = pd.DataFrame( json.loads(
                nl.transform_predict(
                    file=fileName,
                    probability=True,
                    vetting=vetting, hca=hca )) )
            #prediction['text'] = np.NaN # save space... dont need all the txt for this
            if len(prediction) > 0:
                with sql_lock:
                    conn = sqlite3.Connection(prediction_sql)
                    prediction.to_sql("prediction", conn, if_exists='append', chunksize=1000)
                    #print(fileName+"  completed with %d rows" % len(prediction) )
            else:
                print(fileName+" had no results")
        else:
            predictions_t= json.loads(
                nl.transform_predict(
                    file=fileName,
                    probability=True,
                    vetting=vetting, hca=hca ))
            if len(predictions_t) > 0:
                results.put( predictions_t )
        fileNames_q.task_done()
#         except Exception as e:
#             print("error with '%s/%s' -- %s" %(folder,file, e ))
#             fileNames_q.task_done()
#             fileNames_q.put( fileName ) ## add it back to the Queue

#             try:
#                 fileNames_q.task_done()
#                 print(e)
#             except Exception as e2:
#                 print("Error with thread")

import time 
def status_thread():
    while not fileNames_q.empty():
        time.sleep(60)
        print("%s files remain" % fileNames_q.qsize())

### Create a thread pool to parse files ###
running = []
num_work = 8     #T2 xlarge has 8 cores...if mltiproc use 8...
# num_work = 8*2 # go bigger or go home
for i in range(num_work):
    #t=Thread(target=parse_files, args=(fileNames_q, results, sql_lock ) )
    t=Process(target=parse_files, args=(fileNames_q, results, sql_lock ) )
    t.start()
    running.append(t)
# start the 'status thread'
status_thread()

fileNames_q.join()
pred_list = []
while not results.empty():
    temp = results.get()
    pred_list+=temp

### to debug run in a single non-parallel method
# parse_files(fileNames_q, results, sql_lock )

predictions_df = pd.DataFrame(pred_list)
print(len(predictions_df))
print((datetime.now() - start_time).seconds)
predictions_df.head()

creating list of files to run through parrell parsing 
170 files to parse 
../OBX_8_17/ORU-OBX_All_Campuses/HCA/ParsedOBXFiles/HourlyOBX/EDHLungDirectData_Hourly_obx_201802211330.txt had no results
48 files remain
0 files remain
0
120


""


# number of records, unique patients, and vetted records (per dir)

In [4]:
# We are setting up data in the folders. I want the system to flag for size/location/growth 
# from that. I would like it make a list of all reports with nodules that have a size of 6mm 
#     or greator.
# We can benchmark our system vs industry leaders with this data set FYI.


# prediction_sql = "predictions_20180809.sqlite"
# conn = sqlite3.Connection(prediction_sql)
import matplotlib.pyplot as plt
%matplotlib inline  
larger_df = pd.read_sql("""SELECT * FROM prediction   
                """,   conn)
predictions_df = larger_df
predictions_df['directory'] = predictions_df[~pd.isnull(predictions_df.filename)].filename.apply(lambda path: "/".join(path.split("/")[:-1]) )

print("the output predictions have a length of %s rows" % predictions_df.shape[0])
print( "  and the prediction counts:" )
print(predictions_df.prediction.value_counts())

the output predictions have a length of 669 rows
  and the prediction counts:
0    428
1    240
       1
Name: prediction, dtype: int64


## basic output counts

In [6]:
p_df = predictions_df.groupby(['pid']).agg({
               'prediction': lambda vals : "".join([ max( [i for i in set(vals) ]) ][0]),
                'max_nodule_location' : lambda vals : "".join( [i for i in set(vals) ] ),
                'max_nodule_lung' : lambda vals : "".join( [i for i in set(vals) ] ),
                'max_nodule_size' : lambda vals : "".join( [str(i) for i in vals ] ),
                'directory':lambda val: len(val)
                                           }  ).reset_index()

def basic_count(val):
    num_present = [x!="" and x!="0.0" for x in val].count(True)
    num_blank = [x!="" and x!="0.0" for x in val].count(False)
    return "%d present, %d empty" % (num_present,num_blank)
p_df['num_PIDs'] = p_df['prediction']
print(p_df.groupby(['prediction']).agg( {     
                                'num_PIDs' : lambda val: len(val),
                                'directory' : lambda val: sum(val),
#                                 'max_nodule_location' : lambda val: basic_count(val) , 
                                'max_nodule_lung' : lambda val: basic_count(val) , 
                                'max_nodule_size' : lambda val: basic_count(val) ,
                                } ).to_string())

            num_PIDs  directory        max_nodule_lung        max_nodule_size
prediction                                                                   
                   1          1     0 present, 1 empty     0 present, 1 empty
0                371        428  331 present, 40 empty  351 present, 20 empty
1                207        240  171 present, 36 empty  195 present, 12 empty


In [6]:
p_df = predictions_df.groupby(['pid']).agg({
               'prediction': lambda vals : "".join([ max( [i for i in set(vals) ]) ][0]),
                'max_nodule_location' : lambda vals : "".join( [i for i in set(vals) ] ),
                'max_nodule_lung' : lambda vals : "".join( [i for i in set(vals) ] ),
                'max_nodule_size' : lambda vals : "".join( [str(i) for i in vals ] ),
                'directory':lambda val: len(val)  }  ).reset_index()
def basic_count(val):
    num_present = [x!="" and x!="0.0" for x in val].count(True)
    num_blank = [x!="" and x!="0.0" for x in val].count(False)
    return "%d present, %d empty" % (num_present,num_blank)
p_df['num_PIDs'] = p_df['prediction']
print(p_df.groupby(['prediction']).agg( {     
                                'num_PIDs' : lambda val: len(val),
                                'directory' : lambda val: sum(val),
                                'max_nodule_location' : lambda val: basic_count(val) , 
                                'max_nodule_lung' : lambda val: basic_count(val) ,
                                'max_nodule_size' : lambda val: basic_count(val) ,
                                } ).to_string())

            num_PIDs  directory    max_nodule_location        max_nodule_lung        max_nodule_size
prediction                                                                                          
                   1          1     0 present, 1 empty     0 present, 1 empty     0 present, 1 empty
0                371        428  325 present, 46 empty  330 present, 41 empty  351 present, 20 empty
1                207        240  169 present, 38 empty  171 present, 36 empty  195 present, 12 empty


In [7]:
# 386 / (386+192)   # 0-negative  0.6678200692041523
# 192  / (386+192)  #1-pos        0.33217993079584773

In [8]:
# 374/(374+204) #0-negative    0.6470588235294118
# 204/(374+204) #1-pos         0.35294117647058826

In [9]:
# predictions_df[predictions_df.filename=='../OBX_8_17/ORU-OBX_All_Campuses/HCA/ParsedOBXFiles/HourlyOBX/EDHLungDirectData_Hourly_obx_201801261700.txt']

# output text and prediction for spot-analysis
#### currently looking for neg predictions with no nodule-size extracted

In [17]:
obx_head = ['Record Number', 'Facility_Mnemonic_CS', 'MRN', 'URN', 'PAT_ACCT_NUM', 'Observation_text']

def get_patient_text(file, pid_in):
    import warnings
    warnings.simplefilter("ignore")
    grouped_by_patient = pd.read_csv(file, names=obx_head, sep="|", dtype=np.object  )
              #### process data ###
    grouped_by_patient.fillna(value='', inplace=True)
    # PAT_ACCT_NUM
    grouped_by_patient = grouped_by_patient.groupby( by=['Record Number', 'Facility_Mnemonic_CS', 
                                                         'MRN', 'URN', 'PAT_ACCT_NUM']  ).agg({
                     'Observation_text': lambda x: " ".join([str(i).replace("^","\n") for i in set(x) ]),
                                }).reset_index()
    if pid_in not in list(grouped_by_patient.PAT_ACCT_NUM.unique()):
        print("PID:%s not in file...gracefully passing" % pid_in)
    text_out = " ".join(grouped_by_patient[grouped_by_patient.PAT_ACCT_NUM==pid_in]['Observation_text'].values)
    return text_out

for index,row in predictions_df[ (predictions_df.prediction=="0") &\
#                (predictions_df.max_nodule_lung=="") &\
              (predictions_df.max_nodule_size=="0.0")][['filename','evidence', 'is_nodule',
       'max_nodule_change', 'max_nodule_location', 'max_nodule_lung',
       'max_nodule_size', 'nodule_location', 'phrase_counter', 'pid',
       'prediction', 'prev_max_date', 'probability', 'report_date', 'truth_marking']].iterrows():
    print("~~~~~~~~~~~~~~~~~~~~~~%s~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"% row.pid)
    print( "%s\n%s\n\n" % (row,get_patient_text(row.filename,row.pid)) )
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    

~~~~~~~~~~~~~~~~~~~~~~E40010066840~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
filename               ../OBX_8_17/ORU-OBX_All_Campuses/HCA/ParsedOBX...
evidence               comparison: 6 9 findings: chest: lungs: there ...
is_nodule                                                           None
max_nodule_change                                                    0.0
max_nodule_location                                                     
max_nodule_lung                                                    right
max_nodule_size                                                      0.0
nodule_location                                                     None
phrase_counter                                                      None
pid                                                         E40010066840
prediction                                                             0
prev_max_date                                                 03/18/2018
probability                                                 

filename               ../OBX_8_17/ORU-OBX_All_Campuses/HCA/ParsedOBX...
evidence               findings: ct abdomen: the included lung bases ...
is_nodule                                                           None
max_nodule_change                                                    0.0
max_nodule_location                                      left lower lobe
max_nodule_lung                                                     left
max_nodule_size                                                      0.0
nodule_location                                                     None
phrase_counter                                                      None
pid                                                         W00051011908
prediction                                                             0
prev_max_date                                                 08/03/2018
probability                                                        0.818
report_date                                        

In [13]:
predictions_df.columns

Index(['index', 'directory', 'evidence', 'filename', 'is_nodule',
       'max_nodule_change', 'max_nodule_location', 'max_nodule_lung',
       'max_nodule_size', 'nodule_location', 'nodule_lung', 'nodule_phrase',
       'nodule_size', 'nodule_size_numeric', 'phrase_counter', 'pid',
       'prediction', 'prev_max_date', 'probability', 'report_date', 'text',
       'truth_marking'],
      dtype='object')

In [29]:
# EDHLungDirectData_Hourly_obx_201805231144.txt
# E40010341604 
# --- groundglass centrilobular nodules...

In [ ]:
#### end reading

In [ ]:

predictions_df['directory'] = predictions_df[~pd.isnull(predictions_df.filename)].filename.apply(lambda path: "/".join(path.split("/")[:-1]) )
predictions_df.groupby(['directory']).agg({'max_nodule_size':'count',
                                       'pid': lambda x: len(list(set(x)) ),
                                       'truth_marking': lambda vals : len([v for v in vals if v==1 or v ==0])}  )

# number of records and unique patients with nodule size (per dir)

In [27]:
predictions_df[predictions_df.max_nodule_size != 0].groupby(['directory'])\
    .agg({'max_nodule_size':'count',
         'pid': lambda x: len(list(set(x)) ) }  )

,max_nodule_size,pid
directory,,
,1,1
../OBX_8_17/ORU-OBX_All_Campuses/HCA/ParsedOBXFiles/HourlyOBX,668,578


# count of nodule locations by records and unique patients (per dir) 

In [ ]:
predictions_df[predictions_df.max_nodule_location != ''].groupby(['directory'])\
    .agg({'max_nodule_location':'count',
         'pid': lambda x: len(list(set(x)) ) }  )

In [ ]:
# breakdown of location counts
predictions_df[predictions_df.max_nodule_location != ''].groupby(['directory','max_nodule_location'])\
    .agg({'filename':'count',
         'pid': lambda x: len(list(set(x)) ) }  )

In [ ]:
# number of 'Truth' rows
predictions_df[predictions_df.evidence !=""].shape

In [ ]:
# count of patiants where growth was recorded and details
growth_huh = predictions_df[predictions_df.max_nodule_size > 0.0].groupby(['directory','pid'])\
    .agg( {'max_nodule_size': 'unique',
          } ).reset_index()
growth_huh['sizehuh'] =growth_huh.max_nodule_size.apply(lambda x : len(set(x) ))
growth_huh = growth_huh[growth_huh['sizehuh'] >1]
growth_huh

In [ ]:
# of patients with growth per source
growth_huh.groupby('directory').agg({'pid': lambda pat: len(list(set(pat))) })

# number of patients with growth

In [ ]:

growth_huh.shape

# number of patients with history per directory/source

In [ ]:
history_df = predictions_df
history_df['num_reportDate'] = history_df['report_date']
history_df = history_df.groupby(['directory','pid'])\
    .agg( {'num_reportDate': 'unique' ,
          'report_date': ['min', 'max'],
          } ).reset_index()
    
history_df['sizehuh'] =history_df.num_reportDate.unique.apply(lambda x : len(set(x) ))
history_df = history_df[history_df['sizehuh'] >1]
print("reports for patients on multiple days only for %s patients in phHealth " % len(history_df))
print("table below shows number of days those patients were seen")
display( history_df.sort_values(by='sizehuh', ascending=False) )

# calculate model accuracy and phrase accuracy
## records positively labeled
### truth_marking of 1 means there was a match on "qb_y" or "[lngnod]" or "afnrzga19b"
## records labeld as false
### truth_marking of 0 means there was a match on "qb_n"
### truth_marking of -1 for all other records with 'is_nodule', but no validation

In [ ]:
predictions_df.truth_marking.value_counts()

# Classification Reports

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import classification_report

def print_model_performance(predictions_df, dir_name='All'):
    dict_of_vals = {}
    output_str = ""
    if dir_name:
        output_str += "Classification results for directory: {}\n\n".format(dir_name)
    if dir_name != 'All':
        predictions_df = predictions_df[(predictions_df.directory == dir_name)]
    marked_negative = predictions_df.query("truth_marking == 0")
    marked_positive = predictions_df.query('truth_marking == 1')
    unmarked = predictions_df.query('truth_marking == -1')
    vetted = predictions_df.query("truth_marking == 0 | truth_marking == 1")
    with_size = predictions_df.query("max_nodule_size > 0")
    with_location = predictions_df.query("max_nodule_location != ''")
    with_lung = predictions_df.query("max_nodule_lung != ''")
    with_growth = predictions_df.query("max_nodule_change != 0")
    predicted_negative = predictions_df.query("prediction == 0")
    predicted_positive = predictions_df.query("prediction == 1")
    dict_of_vals['Negative'] = len(marked_negative)
    dict_of_vals['Positive'] = len(marked_positive)
    dict_of_vals['Unmarked'] = len(unmarked)
    dict_of_vals['pred_negative'] = len(predicted_negative)
    dict_of_vals['pred_positave'] = len(predicted_positive)
    
    dict_of_vals['with_size'] = len(with_size)
    dict_of_vals['with_location'] = len(with_location)
    dict_of_vals['with_lung'] = len(with_lung)
    dict_of_vals['with_growth'] = len(with_growth)
    # output to string out
    output_str += "Number marked negative: {}\n".format(len(marked_negative))
    output_str += "Number marked positive: {}\n".format(len(marked_positive))
    output_str += "Number unmarked: {}\n".format(len(unmarked))
    
    output_str += "\nNumber predicted negative: {}\n".format(len(predicted_negative))
    output_str += "Number predicted positive: {}\n".format(len(predicted_positive))
    
    output_str += "\nNumber with size extracted: {}\n".format(len(with_size))
    output_str += "Number with location extracted: {}\n".format(len(with_location))
    output_str += "Number with lung (right/left) extracted: {}\n".format(len(with_lung))
    output_str += "Number with growth extracted: {}\n\n".format(len(with_growth))
    if len(marked_negative) > 0:
        output_str += "Percent predicted correctly when marked negative: {}\n".format(
            len(marked_negative.query("prediction == 0")) / len(marked_negative))
    if len(marked_positive) > 0:
        output_str += "Percent predicted correctly when marked positive: {}\n".format(
            len(marked_positive.query("prediction == 1")) / len(marked_positive))
    if len(unmarked) > 0:
        output_str += "Predictions when unmarked:\n{}\n".format(unmarked['prediction'].value_counts())
    if not vetted.empty:
        vetted = predictions_df.query("truth_marking == 0 | truth_marking == 1")
        output_str += "\nClassification Report based off of Vetted data\n"
        original_report = classification_report(vetted['prediction'], vetted['truth_marking'] )
        output_str += original_report
    output_str += "\n\n"
    print(output_str)    
    return output_str,dict_of_vals
output_str = ""
t_str,temp_dict=print_model_performance(predictions_df)
data_stats_dict = {'all':temp_dict}
output_str += t_str
for directory in predictions_df['directory'].unique():
    temp_str,temp_dict = print_model_performance(predictions_df, directory)
    data_stats_dict[directory] = temp_dict
    output_str += temp_str
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d")
filename = "model_performance_{}.txt".format(timestamp)
with open(filename, 'w') as f:
    f.write(output_str)
print("Created file {}".format(filename))

In [ ]:
a.to_json()

In [ ]:
### make charts for each 'rating ###
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
a = pd.DataFrame(data_stats_dict).T
marked_columns = ['Negative','Positive', 'Unmarked']
predic_columns = ['pred_negative','pred_positave' ]
counts = ['with_growth','with_location', 'with_lung','with_size']
for i in a.index:
    a.loc[i][counts].plot(kind="bar",title="data counts for %s"% i)
    plt.show()
    
    a.loc[i][marked_columns].plot(kind="pie",title="vetting for %s"% i)
    plt.show()
    
#     plt.hist(a.loc[i][marked_columns], normed=True, alpha=0.5)
#     plt.hist(a.loc[i][predic_columns], normed=True, alpha=0.5)
#     plt.show()

In [ ]:
import warnings
warnings.filterwarnings('ignore')
for di in predictions_df.directory.unique():
    try:
        temp_df = predictions_df[predictions_df.directory == di]
        if len(temp_df) == 0:
            print(" :( sorry no data to assess")
        else:
            print("calculating for %s " % di)
            temp_df.truth_marking = temp_df[~pd.isnull(temp_df.truth_marking) ]['truth_marking'].astype(int)
            print("Classification Report based off of Vetted data for DIRECTORY %s" % di)
            original_report = classification_report(temp_df[~pd.isnull(temp_df.truth_marking) ]['prediction'], 
                                    temp_df[~pd.isnull(temp_df.truth_marking) ]['truth_marking'] )
            print(original_report)
            print()
    except Exception as e:
        print("Error with %s passing over (error:%s)" % (di, e) )


# count of phrases

In [ ]:
predictions_df['evidence'].value_counts()

# count stuff from sql

In [ ]:
conn = sqlite3.Connection("predictions_20180729.sqlite")
pd.read_sql("SELECT COUNT(*) as COUNT FROM prediction", conn)
# patient_data = pd.read_sql("SELECT * FROM patient_data", nl.conn)